# ECDH - *Elliptic Curve Diffie-Hellman*

O acordo de chaves *Diffie-Hellman* sobre curvas elípticas (**ECDH**) é um protocolo análogo ao desenvolvido num dado grupo multiplicativo $Z^*_p$.

## Algoritmo de troca de chaves
Consideremos duas entidades **A, B** que pretendem acordar um segredo comum.
Consideremos ainda que, previamente ao processo de seguida descrito, estas duas entidades acordaram os parâmetros que definem a curva elíptica e definiram um ponto (da curva) $Q$ de ordem $N$.

1. A gera, aleatoriamente, $k_A \epsilon Z^*_N$, que será a sua chave privada e calcula a sua chave pública $[k_A]Q$
2. Analogamente, B gera, aleatoriamente, a sua chave privada $k_B \epsilon Z^*_N$ e calcula a sua chave pública $[k_B]Q$
3. A e B trocam as chaves públicas entre si tal que A recebe $([k_B]Q)$ e B recebe $([k_A]Q)$
4. A calcula $[k_A]([k_B] Q) \equiv [k_A*k_B]Q$
5. B calcula $[k_A]([k_B] Q) \equiv [k_A*k_B]Q$


Note-se que uma terceira entidade que intercepte $[k_A]Q$ e $[k_B]Q$ não será capaz de descobrir nem $k_A$, nem $k_B$, sem resolver o problema do logaritmo discreto (*hard problem*).

## Implementação

A seguinte classe Python implementa o protocolo **ECDH**, usando curvas elípticas sobre corpos binários, com as seguintes restrições:

- a dimensão $n$ do corpo $K = GF(2^{n})$ é fornecida como parâmetro de inicialização da classe;

- a curva é definita pelas raízes em $K^{2}$ de um polinómio $\phi \equiv y^{2} + xy + x^{3}+x^{2}+b$, sendo o parâmetro $b$ escolhido de forma a que a curva $E/\phi$ tenha um grupo de torsão de ordem prima e de tamanho $\geq 2^{n-1}$.

In [6]:
class ECDH:
    def __init__(self, n):
        self.genCurve(n)
        
    def genCurve(self, n):
        K = GF(2^n)
        while True:
            b=0
            while b==0: #E cannot be a singular curve
                b = K.random_element()
            E = EllipticCurve(K,[1,1,0,0,b])
            e_ord = E.order()
            e_fact = list(factor(e_ord))[-1][0]
            if e_fact >= (2^(n-1)):
                while True:
                    P = E.random_element()
                    p_ord = P.order()
                    if p_ord > e_fact:
                        h = Integer(p_ord/e_fact) #cofator h 
                        Q = h * P
                        self.Q = Q
                        self.N = e_fact
                        return E

    def genPrivPubKeyPair(self):
        k = randint(0, self.N-1)
        kQ = k*self.Q
        return k,kQ

    def genSharedKey(self, k, sQ):
        return k*sQ

## Alice

In [7]:
def Alice(conn, ecdh):
    k, kQ = ecdh.genPrivPubKeyPair()
    conn.send(kQ)
    sQ = conn.recv()
    sKey = ecdh.genSharedKey(k, sQ)
    print("[Alice] Shared key: " + str(sKey))

## Bob

In [8]:
def Bob(conn, ecdh):
    s, sQ = ecdh.genPrivPubKeyPair()
    kQ = conn.recv()
    conn.send(sQ)
    sKey = ecdh.genSharedKey(s, kQ)
    print("[Bob] Shared key: " + str(sKey))

In [9]:
from multiprocessing import Process, Pipe
from getpass import getpass

class Connection:
    def __init__(self, left, right, timeout=None):
        left_end, right_end = Pipe()
        self.timeout = timeout
        ecdh = ECDH(5)
        self.lproc = Process(target = left, args=(left_end, ecdh))
        self.rproc = Process(target = right, args=(right_end, ecdh))
        self.left = lambda : left(left_end)
        self.right = lambda : right(right_end)
        
    def auto(self, proc=None):
        if proc == None:
            self.lproc.start()
            self.rproc.start()
            self.lproc.join(self.timeout)
            self.rproc.join(self.timeout)
        else:
            proc.start()
            proc.join(self.timeout)
    def manual(self):
        self.left()
        self.right()

Como se pode observar, a chave gerada por ambas as entidades são equivalentes.

In [10]:
Connection(Alice, Bob, timeout=10).auto()

[Bob] Shared key: (z5^4 + z5^2 + z5 + 1 : z5 + 1 : 1)
[Alice] Shared key: (z5^4 + z5^2 + z5 + 1 : z5 + 1 : 1)
